# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [125]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sqlalchemy import create_engine
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import RandomizedSearchCV, train_test_split

%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beachac\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\beachac\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\beachac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

### 2. Write a tokenization function to process your text data

In [194]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english')) 
pattern = re.compile(r'[^a-zA-Z0-9]')

def tokenize(text):    
    text = pattern.sub(' ', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w, pos='v') for w in tokens if w not in stop_words]    
   # tokens = [stemmer.stem(w) for w in tokens]
    
    return tokens

In [181]:
ind = 10
print(tokenize(X.to_list()[ind]))
print(X[ind])

['There', 'nothing', 'eat', 'water', 'starve', 'thirsty']
There's nothing to eat and water, we starving and thirsty.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [192]:
vect = CountVectorizer(tokenizer=tokenize, token_pattern=None, lowercase=True)

In [193]:
start=time.time()
test = vect.fit_transform(X)
end=time.time()
print((end - start))

5.91329288482666


In [208]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None, lowercase=True)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(MultinomialNB())),
])

param = {
    'vect__ngram_range': [(1,1), (1,2), (2,2)],
    'vect__max_df':[0.8, 1.0],
    'tfidf__norm': ['l1', 'l2'],
    'tfidf__use_idf': [True, False],
    'clf__estimator__alpha': (0.5, 0.8, 1.0), 
}

cv = GridSearchCV(pipeline,
                  param,
                  scoring=None,
                  verbose=5,
                  error_score='raise',
                  return_train_score=False,
                  n_jobs=-1,
                 # random_state=41
                 )


In [206]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, random_state=41)

In [209]:
start = time.time()
cv.fit(X_train, y_train)
end = time.time()
total = (end-start)/60
print(f'Complete in {total} minutes.')

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Complete in 4.910398288567861 minutes.


In [ ]:
pd.DataFrame(cv.cv_results_).sort_values('rank_test_score').head()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.